<a href="https://colab.research.google.com/github/Theonimfi/Text-mining/blob/main/Testing_Text_Mining_Check_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !tar -xvf  '/content/drive/MyDrive/Data Science and AI/Text mining/enwiki20220701-stripped.tgz' -C '/content/drive/MyDrive/Data Science and AI/Text mining/'

In [ ]:
# Do not uncomment the below 

# ! pip install pynvml
# ! git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# ! python rapidsai-csp-utils/colab/env-check.py
# ! bash rapidsai-csp-utils/colab/update_gcc.sh
# import os
# os._exit(00)
# import condacolab
# condacolab.install()
# import condacolab
# condacolab.check()
#!python rapidsai-csp-utils/colab/install_rapids.py stable

In [ ]:
import json

# Get the data from My drive
file1 = open('/content/drive/MyDrive/Data Science and AI/Text mining_Shared/enwiki20220701-stripped/AB/wiki_63', 'r')
Lines = file1.readlines()
  
content = []
count = 0
# Strips the newline character
for line in Lines:
    content.append(json.loads(line.split('\n')[0]))

In [ ]:
import pandas as pd 

# Add data to a pandas dataframe
df = pd.DataFrame(content)

In [ ]:
(df['text'].values == '').sum() 

Play with the data

In [ ]:
# Import libraries and download example data
import matplotlib.pyplot as plt
import numpy as np
import string
import nltk
nltk.download('stopwords')
import re
from nltk.tokenize import word_tokenize

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append('of')

#defining the function to remove punctuation except dot (.)
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation or i == '.'])
    return punctuationfree

def tokenization(text):
    tokens = re.split('W+',text)
    return tokens

#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

In [ ]:
# Keep only rows with non empty text
df_new = df[df.text != '']
df_new['text_preprocessed'] = df_new['text']

# Reindexing
df_new.index = range(len(df_new))

In [ ]:
# Preprocessing of text
df_new['text_preprocessed']= df_new['text_preprocessed'].apply(lambda x: remove_punctuation(x))
nltk.download('punkt')
df_new['text_preprocessed']= df_new['text_preprocessed'].apply(lambda x: word_tokenize(str(x)))
df_new['text_preprocessed']= df_new['text_preprocessed'].apply(lambda x: remove_stopwords(x))
df_new['text_preprocessed'] = [' '.join(map(str, l)) for l in df_new['text_preprocessed']]

In [ ]:
df_new.index = range(len(df_new))
df_new.text_preprocessed[0]

In [ ]:
import spacy
from spacy import displacy
from itertools import *
import datetime

# Calculate time spend
start = datetime.datetime.now()
 
spacy.require_gpu()
counter = 0
keep_docs = []
nlp = spacy.load('en_core_web_sm')

# Save in a list all the documents that contain more than one person
for textt in df_new.text_preprocessed:
  counter +=1
  if counter % 10000 == 0:
    end = datetime.datetime.now()
    print("We are in ", counter, "out of ", len(df_new.text_preprocessed))
    print("Took:", (end-start), "time")
  doc = nlp(textt)
  people = []
  # Get the people entities included in a doc 
  for ent in doc.ents:
    if ((ent.label_ == 'PERSON') and (ent.text not in people)):
      people.append(ent.text)
  if len(people) > 1:
    keep_docs.append(doc)

In [ ]:
print(len(keep_docs))

In [ ]:
# Old way, takes a lot of time
# import spacy
# from spacy import displacy
# from itertools import *
# import datetime

# start = datetime.datetime.now()

# # Join text per 2000 document
# texts = []
# for i in range(0, len(df_new),2000):
# 	texts.append(" ".join(text for text in df_new.text_preprocessed[i:i+2000]))
 
# spacy.require_gpu()
# total_entities = []
# counter = 0
# # keep_sentences = []
# keep_docs = []
# nlp = spacy.load('en_core_web_sm')
# nlp.max_length = 100000000
# people = []
# for textt in df_new.text_preprocessed:
#   counter +=1
#   if counter % 10000 == 0:
#     end = datetime.datetime.now()
#     print("We are in ", counter, "out of ", len(df_new.text_preprocessed))
#     print("Took:", (end-start), "time")
#   doc = nlp(textt)
#   for sent in doc.sents:
#     count_people = 0
#     sentence_entities = []
#     # Extract all entity labels from a given doc and add them in key value pairs ie "PERSON": [Barack Obama, Michelle Obama]
#     entities = {key: list(set(map(lambda x: str(x), g))) for key, g in groupby(sorted(sent.ents, key=lambda x: x.label_), lambda x: x.label_)}
#     if 'PERSON' in entities:
#       if len(entities['PERSON']) > 1:
#         keep_sentences.append(sent)


In [ ]:
# # Second option, if we save all the doc_entities in a list (takes also a lot of time)
# keep_sentences = []
# total_count = 0
# for entity in total_entities:
#   total_count += 1
#   print("We are in ",total_count, "out of", len(total_entities)) 
#   for sentence in entity['sentences']:
#     people_in_sentence = sum(person in str(sentence) for person in entity['PERSON'])
#     if people_in_sentence > 1:
#       keep_sentences.append(str(sentence))

In [ ]:
for doc in keep_docs:
  for sent in doc.sents:
    test = str(sent)
    break;
  break;

In [ ]:
displacy.render(nlp(str(test)), jupyter=True, style='ent')

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large',device=0)

In [ ]:
print(keep_sentences[152])

In [ ]:
sentences = keep_sentences
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets
for sentence in sentences:  

  sentence = str(sentence)
  # We need to use the tokenizer manually since we need special tokens.
  extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(sentence,
                                                                              return_tensors=True, return_text=False)[0]["generated_token_ids"]])
  extracted_triplets = extract_triplets(extracted_text[0])
  print(extracted_triplets)